In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import os
import time
import torchvision
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader

import matplotlib.pyplot as plt 
import numpy as np 
import torch.nn.functional as F
from torch.nn import Parameter
import numpy as np

In [2]:
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
print(device)

cuda:1


In [3]:
# path

trainset_path = '/dataset/casia_webface_imgs_112_112'
testset_path = '/dataset/LFW/lfw'

# training dataset preprocessing

trainset_preprocess = transforms.Compose([
    transforms.Resize((112,112)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

# test dataset preprocessing

testset_preprocess = transforms.Compose([
    transforms.ToTensor()
])

trainset = ImageFolder(trainset_path, transform=trainset_preprocess)
testset = ImageFolder(trainset_path, transform=testset_preprocess)
train_loader = DataLoader(trainset, batch_size=64, shuffle=True, num_workers=1)
test_loader = DataLoader(testset, batch_size=64, shuffle=True, num_workers=1)

In [4]:
# training set

TRAIN_NUM_CLASS = len(trainset.classes)
TRAIN_NUM_DATA = len(trainset)

# test set

TEST_NUM_CLASS = len(testset.classes)
TEST_NUM_DATA = len(testset)

In [5]:
class BasicBlock(nn.Module):
    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()

        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes) 

        
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes) 

        self.shortcut = nn.Sequential() # identity
        if stride != 1: 
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes)
            )

    def forward(self, x):
        out = self.conv1(x)
        out = self.bn1(out)
        out = F.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        out += self.shortcut(x)
        out = F.relu(out)
        return out


class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes):
        super(ResNet, self).__init__()
        self.in_planes = 32
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(32)
        self.prelu = nn.PReLU(32)
        self.layer1 = self._make_layer(block, 32, num_blocks[0], stride=1) #32
        self.layer2 = self._make_layer(block, 64, num_blocks[1], stride=2) #16
        self.layer3 = self._make_layer(block, 128, num_blocks[2], stride=2) #8
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.dropout = nn.Dropout(p=0.4, inplace=True)

        #self.weight = Parameter(torch.FloatTensor(num_classes, 64))
        #nn.init.xavier_uniform_(self.weight)

        self.linear = nn.Linear(128, num_classes, bias=False) 
        #with torch.no_grad():
        #  self.linear.weight.div_(torch.norm(self.linear.weight, dim=1, keepdim=True)) #normalize weights 


    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes
        return nn.Sequential(*layers)

    def forward(self, x, label):
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.prelu(out)
        out = F.relu(out)

        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.avgpool(out)
        out = out.view(out.size(0), -1)
        out = self.dropout(out)

        out = self.linear(out) 
        return out

In [6]:
def resnet38():
    return ResNet(BasicBlock, [6, 6, 6], num_classes=TRAIN_NUM_CLASS)

In [7]:

net = resnet38()
net = net.to(device)

learning_rate = 0.1
file_name = 'resnet38_softmax.pth' ###

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=learning_rate)
#momentum=0.9, weight_decay=0.0002

train_losses = []
train_accuracy = [] 
test_losses = []
test_accuracy = []

def train(epoch):
    print('\n[ Train epoch: %d ]' % epoch)
    net.train()
    train_loss = 0
    correct = 0
    total = 0

    for batch_idx, (inputs, targets) in enumerate(train_loader):
        inputs, targets = inputs.to(device), targets.to(device)
        #optimizer.zero_grad()

        outputs = net(inputs, targets)

        lamda = 0.1

        loss = criterion(outputs, targets)
        optimizer.zero_grad()
        loss.backward()

        optimizer.step()

        train_loss += loss.item()
        _, predicted = outputs.max(1)

        total += targets.size(0)
        current_correct = predicted.eq(targets).sum().item()
        correct += current_correct
        
        if batch_idx % 500 == 0:
            print('\nCurrent batch:', str(batch_idx))
            print('Current batch average train accuracy:', current_correct / targets.size(0))
            print('Current batch average train loss:', loss.item() / targets.size(0))
            
            train_losses.append(loss.item() / targets.size(0))
            train_accuracy.append(current_correct / targets.size(0))

    print('\nTotal average train accuarcy:', correct / total)
    print('Total average train loss:', train_loss / total)


def test(epoch):
    print('\n[ Test epoch: %d ]' % epoch)
    net.eval()
    loss = 0
    correct = 0
    total = 0

    for batch_idx, (inputs, targets) in enumerate(test_loader):
        inputs, targets = inputs.to(device), targets.to(device)
        total += targets.size(0)
        #
        zero_targets = torch.zeros(targets.size(0)).to(device)

        outputs, _ = net(inputs, zero_targets)
        loss += criterion(outputs, targets).item()

        _, predicted = outputs.max(1)
        correct += predicted.eq(targets).sum().item()

    print('\nTotal average test accuarcy:', correct / total)
    print('Total average test loss:', loss / total)

    test_losses.append(loss / total)
    test_accuracy.append(correct / total)

In [8]:

def adjust_learning_rate(optimizer, epoch):
    lr = learning_rate
    #if epoch >= 50:
    if epoch >= 50:
        lr /= 10
    #if epoch >= 100:
    if epoch >= 100:
        lr /= 10
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

start_time = time.time()

In [10]:
for epoch in range(0, 120):
    adjust_learning_rate(optimizer, epoch)
    train(epoch)
    #test(epoch)
    print('\nTime elapsed:', time.time() - start_time)


[ Train epoch: 0 ]

Current batch: 0
Current batch average train accuracy: 0.0
Current batch average train loss: 0.13422676920890808

Current batch: 500
Current batch average train accuracy: 0.015625
Current batch average train loss: 0.1352466642856598

Current batch: 1000
Current batch average train accuracy: 0.015625
Current batch average train loss: 0.1325031965970993

Current batch: 1500
Current batch average train accuracy: 0.03125
Current batch average train loss: 0.12945035099983215

Current batch: 2000
Current batch average train accuracy: 0.0
Current batch average train loss: 0.1304677128791809

Current batch: 2500
Current batch average train accuracy: 0.015625
Current batch average train loss: 0.13039493560791016

Current batch: 3000
Current batch average train accuracy: 0.015625
Current batch average train loss: 0.12909387052059174

Current batch: 3500
Current batch average train accuracy: 0.015625
Current batch average train loss: 0.12371207773685455

Current batch: 4000
C

RuntimeError: Caught RuntimeError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/opt/conda/lib/python3.6/site-packages/torch/utils/data/_utils/worker.py", line 202, in _worker_loop
    data = fetcher.fetch(index)
  File "/opt/conda/lib/python3.6/site-packages/torch/utils/data/_utils/fetch.py", line 47, in fetch
    return self.collate_fn(data)
  File "/opt/conda/lib/python3.6/site-packages/torch/utils/data/_utils/collate.py", line 83, in default_collate
    return [default_collate(samples) for samples in transposed]
  File "/opt/conda/lib/python3.6/site-packages/torch/utils/data/_utils/collate.py", line 83, in <listcomp>
    return [default_collate(samples) for samples in transposed]
  File "/opt/conda/lib/python3.6/site-packages/torch/utils/data/_utils/collate.py", line 53, in default_collate
    storage = elem.storage()._new_shared(numel)
  File "/opt/conda/lib/python3.6/site-packages/torch/storage.py", line 137, in _new_shared
    return cls._ne

In [ ]:
# count_parameters
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [ ]:
    state = {
        'net': net.state_dict()
    }

    file_name = 'CNN.pt'
    if not os.path.isdir('checkpoint'):
        os.mkdir('checkpoint')
    torch.save(state, './checkpoint/' + file_name)
    print('Model Saved')